In [ ]:
#import torch
import sklearn

import numpy as np 
import pandas as pd
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px

from sklearn.model_selection import train_test_split 

In [ ]:
import keras_nlp
import keras
import tensorflow as tf

import os
os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

#from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

print(tf.config.list_physical_devices('GPU'))

In [ ]:
from transformers import AutoTokenizer, LlamaModel

In [ ]:
#BASE_PATH = '/kaggle/input/llm-classification-finetuning'
BASE_PATH = '../Data'

In [ ]:
class CFG:
    seed = 42  # Random seed
    preset = "deberta_v3_extra_small_en" # Name of pretrained models
    sequence_length = 256 #512  # Input sequence length
    epochs = 3 # Training epochs
    batch_size = 4  # Batch size
    scheduler = 'cosine'  # Learning rate scheduler
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
    name2label = {v:k for k, v in label2name.items()}
    class_labels = list(label2name.keys())
    class_names = list(label2name.values())

In [ ]:
keras.utils.set_random_seed(CFG.seed)
keras.mixed_precision.set_global_policy("mixed_float16")

## Files

`train.csv`
- `id`
- `model_[a/b]`: Model identity, present in train.csv but not in test.csv.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.
- `winner_model_[a/b/tie]`: Binary columns indicating the judge's selection (ground truth target).

`test.csv`
- `id`: Unique identifier for each row.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.

> !!!! Note that each interaction may have multiple prompts and responses, but this notebook will use only one prompt per interaction. You can choose to use all prompts and responses. Additionally, prompts and responses in the dataframe are provided as string-formatted lists, so they need to be converted to literal lists using eval().

> !!! TODO : use all prompts

In [ ]:
import os

# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

In [ ]:
# Load Train Data
df = pd.read_csv(f'{BASE_PATH}/train.csv') 

In [ ]:
df

## Train Data

In [ ]:
# Sample data
# df = df.sample(frac=0.10)

# Take the first prompt and its associated response
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Label conversion
df["class_name"] = df[["winner_model_a", "winner_model_b", "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)

# Show Sample
df.head()

## Test Data

In [ ]:
# Load Test Data
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')

# Take the first prompt and response
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Show Sample
test_df.head()

## Contextualize Response with Prompt

Creating pair "Prompt/Response" for each response. (e.g., (P + R_A), (P + R_B), etc.).
This approach is similar to the multiple-choice question task in NLP.

>Note that some prompts and responses may not be encoded with utf-8, resulting in errors when creating the dataloader. In such cases, we will replace them with an empty string.

In [ ]:
# Define a function to create options based on the prompt and choices
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        response_b = ""
        row["encode_fail"] = True
        
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # Response from Model A
                        f"Prompt: {prompt}\n\nResponse: {response_b}"  # Response from Model B
                        ]
    return row

In [ ]:
df = df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(df.head(2))  # Display the first 2 rows of df

test_df = test_df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(test_df.head(2))  # Display the first 2 rows of df

### Encoding fails stats

In [ ]:
df.encode_fail.value_counts(normalize=False)

We can see that only 1% of the samples failed to be encoded, while 99% of the samples don't have any issues. A similar pattern can be expected for the test data as well. Thus, considering empty strings for this small portion of the data will not have much impact on our training and inference.

## EDA

In [ ]:
model_df = pd.concat([df.model_a, df.model_b])
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Create a bar plot with custom styling using Plotly
fig = px.bar(counts, x='LLM', y='Count',
                title='Distribution of LLMs',
                color='Count', color_continuous_scale='viridis', width=1000)

fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability

fig.show()

## Winning distribution

In [ ]:
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']

fig = px.bar(counts, x='Winner', y='Win Count',
                title='Winner distribution for Train Data',
                labels={'Winner': 'Winner', 'Win Count': 'Win Count'},
                color='Winner', color_continuous_scale='viridis', width=1000)

fig.update_layout(xaxis_title="Winner", yaxis_title="Win Count")

fig.show()

## Winning distribution per model (todo ratio)

In [ ]:
models_a = df.query('winner_model_a == 1').groupby(['model_a'])['winner_model_a'].count().reset_index() 
models_a.columns = ['model', 'wins']
models_a['losses'] = df.query('winner_model_a == 0').groupby(['model_a'])['winner_model_a'].count().reset_index()['winner_model_a']

models_b = df.query('winner_model_b == 1').groupby(['model_b'])['winner_model_b'].count().reset_index() 
models_b.columns = ['model', 'wins']
models_b['losses'] = df.query('winner_model_b == 0').groupby(['model_b'])['winner_model_b'].count().reset_index()['winner_model_b']

models = models_a
models[['wins', 'losses']] = models_a[['wins', 'losses']] + models_b[['wins', 'losses']]

In [ ]:
models.sort_values(by='wins', ascending=False, inplace=True)

fig = px.bar(
    data_frame = models,
    x = "model",
    y = ["wins","losses"],
    opacity = 0.9,
    #orientation = "v",
    barmode = 'stack',
    title='Wins/losses rate per model',
)

fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability

fig.show()

## Data Split

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])

## Preprocessing

- https://keras.io/keras_hub/api/preprocessing_layers/
- https://keras.io/keras_hub/api/tokenizers/

In [ ]:
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # Name of the model
    sequence_length=CFG.sequence_length, # Max sequence length, will be padded if shorter
)

In [ ]:
outs = preprocessor(df.options.iloc[0])  # Process options for the first row

# Display the shape of each processed output
for k, v in outs.items():
    print(k, ":", v.shape)

In [ ]:
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Preprocess text
    return (text, label) if label is not None else text  # Return processed text and label if available

## DataLoader

In [ ]:
def build_dataset(texts, labels=None, batch_size=32,
                    cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    opt = tf.data.Options()  # Create dataset options
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=False)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds  # Return the built dataset

In [ ]:
# Train
train_texts = train_df.options.tolist()  # Extract training texts
train_labels = train_df.class_label.tolist()  # Extract training labels
train_ds = build_dataset(train_texts, train_labels,
                            batch_size=CFG.batch_size,
                            shuffle=True)

# Valid
valid_texts = valid_df.options.tolist()  # Extract validation texts
valid_labels = valid_df.class_label.tolist()  # Extract validation labels
valid_ds = build_dataset(valid_texts, valid_labels,
                            batch_size=CFG.batch_size,
                            shuffle=False)

## LR Schedule

In [ ]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [ ]:
lr_cb = get_lr_callback(CFG.batch_size, plot=True)

## Model Checkpoint

In [ ]:
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                            monitor='val_log_loss',
                                            save_best_only=True,
                                            save_weights_only=True,
                                            mode='min')  # Get Model checkpoint callback

## Metric

In [ ]:
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")

## Modeling

Our approach utilizes keras_nlp.models.DebertaV3Classifier to process each prompt and response pair, generating output embeddings. We then concatenate these embeddings and pass them through a Pooling layer and a classifier to obtain logits, followed by a softmax function for the final output.

When dealing with multiple responses, we use a weight-sharing strategy. This means we provide the model with one response at a time along with the prompt (P + R_A), (P + R_B), etc., using the same model weights for all responses. After obtaining embeddings for all responses, we concatenate them and apply average pooling. Next, we use a Linear/Dense layer along with the Softmax function as the classifier for the final result. Providing all responses at once would increase text length and complicate model handling. Note that, in the classifier, we use 3 classes for winner_model_a, winner_model_b, and draw cases.

In [ ]:
# Define input layers
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
# Create a DebertaV3Classifier backbone
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)

# Compute embeddings for first response: (P + R_A) using backbone
response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)

# Compute embeddings for second response: (P + R_B), using the same backbone
response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)

# Compute final output
embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)

# Compile the model with optimizer, loss, and metrics
model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

## Trainning

In [ ]:
# Start training the model
history = model.fit(
    train_ds,
    epochs=CFG.epochs,
    validation_data=valid_ds,
    callbacks=[lr_cb, ckpt_cb]
)

## Load Best model

In [ ]:
model.load_weights('/kaggle/working/best_model.weights.h5')

## Prediction

In [ ]:
# Build test dataset
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                            batch_size=min(len(test_df), CFG.batch_size),
                            shuffle=False)

In [ ]:
test_preds = model.predict(test_ds, verbose=1)

## Submission

In [ ]:
sub_df = test_df[["id"]].copy()
sub_df[CFG.class_names] = test_preds.tolist()
sub_df.to_csv("submission.csv", index=False)
sub_df.head()

## Future Directions

In this notebook, we've achieved a good score with a small model and modest token length. But there's plenty of room to improve. Here's how:

- Try bigger models like Deberta-Base or Deberta-Small, or even LLMs like Gemma.
- Increase max token length to reduce loss of data.
- Use a five-fold cross-validation and ensemble to make the model robust and get better scores.
- Add augmentation like shuffling response orders for more robust performance.
- Train for more epochs.
- Tune the learning rate scheduler.